# PREPARACIÓN NOTEBOOK
---

## Librerías

In [1]:
from google.colab import drive
import sys
import os
import sqlite3 as sql #Crear y trabajar bases de datos ligeras
import pandas as pd
from collections import Counter

## Conexión con la BD y el repositorio en Drive

In [2]:
#Conectar al drive local
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

## Conexión al notebook de funciones

In [4]:
%run a_funciones.ipynb

# SQL
---

## Conexión

In [5]:
con = sql.connect('data/db_movies') #Conectarse a la base de datos existente y transportar datos
cur = con.cursor() #Otra conexión (cursor) para ejecutar las consultas en la bd sin traer ni llevar info

In [6]:
#Consultar qué tablas disponibles tiene la bd
cur.execute(""" select name from sqlite_master where type= 'table' """)
cur.fetchall()

[('ratings',), ('movies',)]

## Exploración

In [7]:
tablas_a_explorar = ['ratings', 'movies']
for tabla in tablas_a_explorar:
    explorar_tabla(con, tabla)

------------

Explorando la tabla 'ratings'

Esquema de la tabla 'ratings':
  - Nombre: userId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: movieId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: rating, Tipo: REAL, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: timestamp, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0

Primeras 10 filas de la tabla 'ratings':
(1, 1, 4.0, 964982703)
(1, 3, 4.0, 964981247)
(1, 6, 4.0, 964982224)
(1, 47, 5.0, 964983815)
(1, 50, 5.0, 964982931)
(1, 70, 3.0, 964982400)
(1, 101, 5.0, 964980868)
(1, 110, 4.0, 964982176)
(1, 151, 5.0, 964984041)
(1, 157, 5.0, 964984100)

Número total de filas en 'ratings': 100836
------------

Explorando la tabla 'movies'

Esquema de la tabla 'movies':
  - Nombre: movieId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: title, Tipo: TEXT, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: genres, Tipo: TEXT, ¿Nulo?: 0, Clave Primaria: 0

Primeras 10 filas de la tabla 'movies':
(1, 'Toy Story (1995)', 'Adventure|Anim

## Exploración profunda: Tabla ratings



In [8]:
try:
    # 1. Conteo de usuarios únicos
    cur.execute("SELECT COUNT(DISTINCT userId) FROM ratings;")
    num_usuarios_unicos = cur.fetchone()[0]
    print(f"Número de usuarios únicos: {num_usuarios_unicos}")

    # 2. Medidas estadísticas
    cur.execute("SELECT AVG(rating), MIN(rating), MAX(rating) FROM ratings;")
    promedio_rating, min_rating, max_rating = cur.fetchone()
    print(f"Promedio de rating: {promedio_rating:.2f}")
    print(f"Rating mínimo: {min_rating}")
    print(f"Rating máximo: {max_rating}")

    # 3. Moda
    cur.execute("SELECT rating FROM ratings;")
    ratings = [row[0] for row in cur.fetchall()]
    if ratings:
        moda_rating = calcular_moda(ratings)
        print(f"Moda del rating: {moda_rating}")
    else:
        print("No hay ratings para calcular la moda.")

except sql.Error as e:
    print(f"Error al explorar la tabla 'ratings': {e}")

Número de usuarios únicos: 610
Promedio de rating: 3.50
Rating mínimo: 0.5
Rating máximo: 5.0
Moda del rating: 4.0


## Exploración profunda: Tabla movies

Para esta exploración se usara la librería de pandas ya que permite una manipulación mas rápida y menos engorrosa de la tabla al realizar los split's y extracciones de fragmentos de la cadena de texto mas rápidamente

In [22]:
try:
    # Consultar la tabla 'movies' y cargarla en un DataFrame de Pandas
    query = "SELECT * FROM movies"
    df_movies = pd.read_sql_query(query, con)

    # Extraer los últimos 6 caracteres de la columna 'title' y crear la columna 'Year'
    df_movies['Year'] = df_movies['title'].str[-6:]

    # Limpiar la columna 'Year' removiendo los paréntesis
    df_movies['Year'] = df_movies['Year'].str.replace(r'\(|\)', '', regex=True)

    # Separar los géneros de la columna 'genres'
    df_movies['genres_split'] = df_movies['genres'].str.split('|')

except sql.Error as e:
    print(f"Error: {e}")

In [23]:
df_movies.head()

,movieId,title,genres,Year,genres_split
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,1995,[Comedy]


In [24]:
#Creación de una copia del DF
df_movies1 = df_movies[['movieId', 'title', 'Year', 'genres_split']].copy()
#Eliminación del año en la columna "title"
df_movies1['title'] = df_movies1['title'].str[:-7]
df_movies1.head()

,movieId,title,Year,genres_split
0,1,Toy Story,1995,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,1995,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,1995,"[Comedy, Romance]"
3,4,Waiting to Exhale,1995,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,1995,[Comedy]


In [25]:
#Separación en columnas de los generos

# 1. Obtener todos los géneros únicos de la columna 'genres_split'
all_genres = df_movies1['genres_split'].explode().unique()

# 2. Crear columnas booleanas para cada género único
for genre in all_genres:
    df_movies1[genre] = df_movies['genres_split'].apply(lambda x: 1 if genre in x else 0)

# 3. Eliminar la columna original 'genres_split'
df_movies1 = df_movies1.drop('genres_split', axis=1, errors='ignore')

   movieId                        title  Year  Adventure  Animation  Children  \
0        1                    Toy Story  1995          1          1         1   
1        2                      Jumanji  1995          1          0         1   
2        3             Grumpier Old Men  1995          0          0         0   
3        4            Waiting to Exhale  1995          0          0         0   
4        5  Father of the Bride Part II  1995          0          0         0   

   Comedy  Fantasy  Romance  Drama  ...  Horror  Mystery  Sci-Fi  War  \
0       1        1        0      0  ...       0        0       0    0   
1       0        1        0      0  ...       0        0       0    0   
2       1        0        1      0  ...       0        0       0    0   
3       1        0        1      1  ...       0        0       0    0   
4       1        0        0      0  ...       0        0       0    0   

   Musical  Documentary  IMAX  Western  Film-Noir  (no genres listed)  
0 

In [26]:
df_movies1.head(10)

,movieId,title,Year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1995,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Heat,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Sabrina,1995,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Tom and Huck,1995,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Sudden Death,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye,1995,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# PANDAS

In [13]:
#Crear df's de pandas con las tablas
df_ratings = pd.read_sql('select * from ratings', con)
df_movies = pd.read_sql('select * from movies', con)

## Exploración de df_ratings

In [14]:
#Visualización de df_ratings
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
#Renombrar columnas del df e indexarlo con movie_id
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = df_ratings.set_index('movie_id')
df_ratings

,user_id,rating,timestamp
movie_id,,,
1,1,4.0,964982703
3,1,4.0,964981247
6,1,4.0,964982224
47,1,5.0,964983815
50,1,5.0,964982931
...,...,...,...
166534,610,4.0,1493848402
168248,610,5.0,1493850091
168250,610,5.0,1494273047


In [16]:
#La columna 'fecha_vista' es en realidad un unix timestamp, es decir, el número de segundos desde el 1 de enero de 1970 (UTC). Se restructura.
df_ratings['fecha_vista'] = pd.to_datetime(df_ratings['timestamp'], unit='s')
df_ratings

,user_id,rating,timestamp,fecha_vista
movie_id,,,,
1,1,4.0,964982703,2000-07-30 18:45:03
3,1,4.0,964981247,2000-07-30 18:20:47
6,1,4.0,964982224,2000-07-30 18:37:04
47,1,5.0,964983815,2000-07-30 19:03:35
50,1,5.0,964982931,2000-07-30 18:48:51
...,...,...,...,...
166534,610,4.0,1493848402,2017-05-03 21:53:22
168248,610,5.0,1493850091,2017-05-03 22:21:31
168250,610,5.0,1494273047,2017-05-08 19:50:47


## Exploración de df_movies

In [17]:
#Visualización de df_movies
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
#Renombrar columnas del df e indexarlo por movie_id
df_movies.columns = ['movie_id', 'titulo', 'genero']
df_movies = df_movies.set_index('movie_id')
df_movies

,titulo,genero
movie_id,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama
